In [2]:
import requests

In [3]:
home_adress = "http://0.0.0.0:80"


In [37]:
session = requests.post(
    'http://34.118.55.236:80/start?user_id=alex@example.com&task_id=1'
    # "http://0.0.0.0:80/start?user_id=alex@example.com&task_id=1"
).json()['session_id']

In [38]:
session

'3ee7f24a-9a17-4a51-a8c2-82a59876768d'

In [26]:
import urllib

In [35]:
user_id = "alex@example.com"
task_id = "1"
task_name = "Alex's Task"
# df only first 2 rows
code = "df=df.iloc[:2]"
explanation = "explanation"
# encode sting for the url 
code = urllib.parse.quote(code)
explanation = urllib.parse.quote(explanation)


execute_code = requests.post(
    f"http://0.0.0.0:80/execute?session_id={session}&code={code}&explanation=print(df.head())&user_id={user_id}&task_id={task_id}&task_name={task_name}"
).json()

In [36]:
print(execute_code['code_output'])